## Introduction
In this workflow, we will perform pathway enrichment analysis to differential expressed genes for both diseases on two biopsy locations ileum and rectum.
We will be performing pathway enrichment analysis on a differential gene expression dataset. The dataset compares the expression of transcripts in inflammatory bowel disease (CD and UC) biopsy locations (ileum and rectum) versus healthy people. Differential expression analysis has already been performed (see step 2 of this workflow), generating log2foldchange and (adjusted) p-values data for each gene.

## R environment setup


In [ ]:
# check if libraries are already installed > otherwise install it
if (!requireNamespace("BiocManager", quietly = TRUE)) install.packages("BiocManager")
if(!"rstudioapi" %in% installed.packages()) BiocManager::install("rstudioapi")
if(!"org.Hs.eg.db" %in% installed.packages()) BiocManager::install("org.Hs.eg.db")  
if(!"AnnotationDbi" %in% installed.packages()) BiocManager::install("AnnotationDbi")
if(!"rWikiPathways" %in% installed.packages()) BiocManager::install("rWikiPathways")
if(!"clusterProfiler" %in% installed.packages()) BiocManager::install("clusterProfiler") 
if(!"dplyr" %in% installed.packages()){install.packages("dplyr")}

#loading installed libraries
library(rstudioapi) # interface for interacting with RStudio IDE with R code.
library(org.Hs.eg.db) #This is the organism annotation package ("org") for Homo sapiens ("Hs"), organized as an AnnotationDbi   package ("db"), using Entrez Gene IDs ("eg") as primary key.
library(AnnotationDbi) # for connecting and querying annotation databases
library(rWikiPathways) # for programmatic access to WikiPathways content
library(clusterProfiler) # for implementing methods to analyze and visualize functional profiles of genomic data
library(dplyr)

# set your working environment to the location where your current source file is saved into.
setwd(dirname(rstudioapi::getSourceEditorContext()$path))


## Importing dataset
The data will be read for the disease on two biopsy locations


In [ ]:
## Select a disorder to analyse (options; CD or UC)
disorder <- "CD"
##Obtain data from step 2:
setwd('..')
work_DIR <- getwd()

if (disorder == "CD") {
  dataset_CD <- read.delim("3-identifier_mapping/output/IDMapping_CD.tsv")
  #filter out  unused columns, we select Entrez.ID, log2FC and pvalue, remove NA values: #background genes to be used in enrichment analysis
  dataset <- na.omit(subset( dataset_CD, select = c(2,4:7)))
  print("Selected disorder is Crohn's disease")
}else if(disorder == "UC"){ 
  dataset_UC <- read.delim("3-identifier_mapping/output/IDMapping_UC.tsv")
  #filter out  unused columns, we select Entrez.ID, log2FC and pvalue
  dataset <- na.omit(subset( dataset_UC, select = c(2,4:7)))
  print("Selected disorder is Ulcerative Colitis")}else{print("Disorder not Recognised")
  }

# Set Working Directory back to current folder
setwd(dirname(rstudioapi::getSourceEditorContext()$path))
work_DIR <- getwd()



## Getting differentially expressed genes including up-regulated and down-regulated genes using the criteria


In [ ]:
#if the results folder does not exist create it 
if(!dir.exists("output")) dir.create("output")
#we will use selection criteria as Fold change=1.5,log2FC=0.58 and p.value < 0.05
#for ileum location
up.genes.ileum   <- dataset[dataset$log2FC_ileum >= 0.58 & dataset$pvalue_ileum < 0.05, 1] 
down.genes.ileum <- dataset[dataset$log2FC_ileum <= -0.58 & dataset$pvalue_ileum < 0.05, 1] 
deg.ileum <- unique(dataset[!is.na(dataset$ENTREZ.ID) & !is.na(dataset$pvalue_ileum) & dataset$pvalue_ileum < 0.05 & abs(dataset$log2FC_ileum) > 0.58,c(1:3)])
write.table(deg.ileum,file = paste0("output/DEGs_",disorder,"_ileum"),sep="\t", quote=FALSE, row.names = FALSE)

#for rectum location
up.genes.rectum   <- dataset[dataset$log2FC_rectum >= 0.58 & dataset$pvalue_rectum < 0.05, 1] 
down.genes.rectum <- dataset[dataset$log2FC_rectum <= -0.58 & dataset$pvalue_rectum < 0.05, 1] 
deg.rectum <- unique(dataset[!is.na(dataset$ENTREZ.ID) & !is.na(dataset$pvalue_rectum) & dataset$pvalue_rectum < 0.05 & abs(dataset$log2FC_rectum) > 0.58,c(1,4:5)])
write.table(deg.rectum, file=paste0("output/DEGs_",disorder,"_rectum"),sep="\t", quote=FALSE, row.names = FALSE)


## Pathway Enrichment Analysis
In this section, we will perform pathway enrichment analysis. The clusterProfiler R-package is used to perform over-representation analysis (ORA). The function can be easily replaced to use other enrichment methods (GSEA / rSEA / etc). 


In [ ]:
##Work with local file (for publication), or new download:
pathway_data <- "local" #Options: local, new
if (pathway_data == "local") {
  wp.hs.gmt <-list.files(work_DIR, pattern="wikipathways", full.names=FALSE)
  paste0("Using local file, from: ", wp.hs.gmt )
}else if(pathway_data == "new"){ 
#below code should be performed first to handle the ssl certificate error while downloading pathways 
options(RCurlOptions = list(cainfo = paste0( tempdir() , "/cacert.pem" ), ssl.verifypeer = FALSE))
#downloading latest pathway gmt files for human 
wp.hs.gmt <- rWikiPathways::downloadPathwayArchive(organism="Homo sapiens", format = "gmt")
  paste0("Using new data, from: ", wp.hs.gmt)}else{print("Pathway data type not recognized")
  }

#Now that we have got the latest GMT file for human pathways, 
#we can process it to generate the two dataframes we need for enricher
wp2gene   <- rWikiPathways::readPathwayGMT(wp.hs.gmt)
wpid2gene <- wp2gene %>% dplyr::select(wpid,gene) #TERM2GENE
wpid2name <- wp2gene %>% dplyr::select(wpid,name) #TERM2NAME

##################ILEUM location#######################
# The clusterProfiler R-package is used to perform over-representation analysis (ORA)
# The function can be easily replaced to use other enrichment methods (GSEA / rSEA / etc). 
ewp.ileum <- clusterProfiler::enricher(
  deg.ileum$ENTREZ.ID,# a vector of gene IDs
  universe = as.character(dataset$ENTREZ.ID), #background genes to be used in enrichment analysis
  pAdjustMethod = "fdr",#you can change this to hochberg, bonferronni or none etc.
  pvalueCutoff = 1, #adjusted pvalue cutoff on enrichment tests to report, we set it a wider criteria then we will filter out
  #results based on padjust and qvalue in next section which is enrichment result visualization
  qvalueCutoff = 1, #qvalue cutoff on enrichment tests to report as significant, 
                    #multiple hypothesis testing
  TERM2GENE = wpid2gene, #user input annotation of TERM TO GENE mapping
  TERM2NAME = wpid2name) #user input of TERM TO NAME mapping

ewp.ileum.res <- as.data.frame(ewp.ileum) 

#interpretation of the output: 
#     BgRatio   = (number of genes measured in the current pathway) / (number of genes measured in all pathways)
#     geneRatio = (number of DEGs in the current pathway) / (total number of DEGs in all pathways)

##Print location:
paste0("Pathways enrichment results for disorder: ", disorder , ", location: ILEUM")
# number of genes measured in all pathways
paste0("The number of genes measured in all pathways is: ", length(ewp.ileum@universe))
# number of DEGs in all pathways
paste0("The number of DEGs measured in all pathways is: ", length(deg.ileum$ENTREZ.ID[deg.ileum$ENTREZ.ID %in% unique(wp2gene$gene)]))
#number of enriched pathways
paste0("The number of enriched pathways is: ", num.pathways.ileum <- dim(ewp.ileum.res)[1])

#exporting results to the file
write.table(ewp.ileum.res, file=paste0("output/enrichResults_ORA_",disorder,"_ileum"),
            sep = "\t" ,quote = FALSE, row.names = FALSE)

##################RECTUM location#######################
ewp.rectum <- clusterProfiler::enricher(
  deg.rectum$ENTREZ.ID,
  universe = as.character(dataset$ENTREZ.ID), #background genes to be used in enrichment analysis
  pAdjustMethod = "fdr",#you can change it as BH, bonferronni etc.
  pvalueCutoff = 1, #padjust cutoff
  qvalueCutoff = 1, #q value cutoff 
  TERM2GENE = wpid2gene,
  TERM2NAME = wpid2name)
ewp.rectum.res <- as.data.frame(ewp.rectum) 

##Print location:
paste0("Pathways enrichment results for disorder: ", disorder , ", location: RECTUM")
# number of genes measured in all pathways
paste0("The number of genes measured in all pathways is: ", length(ewp.rectum@universe))
# number of DEGs in all pathways
paste0("The number of DEGs measured in all pathways is: ", length(deg.rectum$ENTREZ.ID[deg.rectum$ENTREZ.ID %in% unique(wp2gene$gene)]))
#number of enriched pathways
paste0("The number of enriched pathways is: ", num.pathways.rectum <- dim(ewp.rectum.res)[1])

#exporting results to the file
write.table(ewp.rectum.res, file=paste0("output/enrichResults_ORA_",disorder,"_rectum"),
            sep = "\t" ,quote = FALSE, row.names = FALSE)

#if the new data file exist, remove it (so it does not conflict with running the code against the local file) 
if(pathway_data == "new") file.remove(wp.hs.gmt)



##Print session info and remove large datasets:


In [ ]:
##Print session info:
sessionInfo()

##Remove data objects which are not needed for further processing:
rm(list=setdiff(ls(), c("ewp.rectum.res", "ewp.ileum.res")))



### Last, we create a Jupyter notebook from this script


In [ ]:
#Jupyter Notebook file
if(!"devtools" %in% installed.packages()) BiocManager::install("devtools")
devtools::install_github("mkearney/rmd2jupyter", force=TRUE)
library(devtools)
library(rmd2jupyter)
setwd(dirname(rstudioapi::getSourceEditorContext()$path))
rmd2jupyter("pathway_analysis.Rmd")
